In [14]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime


In [ ]:
# 读取 CSV 文件
#route = "C:/Users/M1107171/MIMIC/清出來的資料/DNR/20240616"
route = "C:/Users/USER/M1326168/MIMIC/DNR/20241002"    
input_csv = f'{route}/full_step1.csv'  # 替换成实际文件名
output_csv = f'{route}/number_2.5%.csv'  # 输出文件名
boundaries_csv = f'{route}/boundaries.csv'  # 边界信息的 CSV 文件

In [16]:
df = pd.read_csv(input_csv)

In [17]:
integer_columns = df.select_dtypes(include='integer').columns.tolist()
print(integer_columns)

['stay_id', 'use_vent', 'Vasopressor', 'Relaxant', 'Sedation', 'PPI', 'Pain control', 'Aspergillus', 'Candida', 'Abdomen', 'Blood', 'Respiratory tract', 'Skin and soft tissue', 'Urinary tract', 'Others', 'dod', 'dod_3day', 'dod_7day', 'dod_30day', 'dod_60day', 'dod_90day', 'InvasiveVent', 'tracheostomy', 'NonInvasiveVent', 'SupplementalOxygen', 'HFNC', 'Muscle', 'Vocalization', 'Body Movements', 'Facial Expressions', 'CPOT(SUM)', 'dialysis', 'CVICU', 'CCU', 'MICU', 'MICU/SICU', 'Neuro Intermediate', 'Neuro Stepdown', 'Neuro SICU', 'SICU', 'TSICU', 'age', 'dnr', 'apsiii', 'MI', 'CHF', 'PVD', 'CVD', 'Dementia', 'CPD', 'RD', 'PUD', 'MLD', 'DM_acute', 'DM_Chronic', 'Hemiplegia', 'Renal', 'Malignancy', 'LD', 'MST', 'AIDS', 'Weaning', 'Reintubation', 'Weaning_successful']


In [18]:
"""
離群值移除
"""
# 确定哪些列是数值型并避开stay_id
numeric_cols = df.select_dtypes(include=[np.number]).columns[1:]

# 创建一个 DataFrame 来记录边界值
boundaries = pd.DataFrame(columns=['Column', 'Lower_Bound', 'Upper_Bound'])

# 设置 tqdm 进度条
with tqdm(total=len(numeric_cols)) as pbar:  # 创建进度条
    # 遍历所有数值型列
    for col in numeric_cols:
        if col in integer_columns:
            continue
        
        # 找到上下 2.5% 的界限
        lower_quantile = df[col].quantile(0.025)  # 下 2.5% 边界
        upper_quantile = df[col].quantile(0.975)  # 上 2.5% 边界
        
        # 将小于下界的值设置为下界
        df.loc[df[col] < lower_quantile, col] = lower_quantile
        
        # 将大于上界的值设置为上界
        df.loc[df[col] > upper_quantile, col] = upper_quantile
        
        # 记录边界值到边界 DataFrame 中
        new_boundary = pd.DataFrame({
            'Column': [col],
            'Lower_Bound': [lower_quantile],
            'Upper_Bound': [upper_quantile]
        })

        boundaries = pd.concat([boundaries, new_boundary], ignore_index=True)
        
        # 更新进度条
        pbar.update(1)  # 更新进度条的进度



## 将边界 DataFrame 行列互換後保存到 CSV 文件
#transposed_boundaries = boundaries.transpose()  # 互換行與列
#transposed_boundaries.to_csv(boundaries_csv, header=False)  # 保存到 CSV，不輸出標題行

 70%|██████▉   | 146/209 [00:00<00:00, 379.45it/s]


In [19]:
"""
衍生特徵 - Full_code_interval
"""
def str_to_date(date_string):
    #date_string = "2024-05-21"
    date_format = "%Y-%m-%d %H:%M:%S"
    date_object = datetime.strptime(date_string, date_format)
    #print("日期对象:", date_object)
    return date_object
    
# 創建一個新的列來存儲Full code間隔天數
df['Full_code_interval'] = 0

# 遍歷每個 stay_id 分組
for stay_id, group in df.groupby('stay_id'):
    last_Full_code_date = None
    for idx, row in group.iterrows():
        if row['Full code'] == 1:  # 假設Full code發生時Full code列的值為1
            df.loc[idx, 'Full_code_interval'] = 0
            last_Full_code_date = row['date']
        elif last_Full_code_date is not None:
            #df.loc[idx, 'Full_code_interval'] = (row['date'] - last_Full_code_date).days
            df.loc[idx, 'Full_code_interval'] = (str_to_date(row['date']) - str_to_date(last_Full_code_date)).days            
        else:
            df.loc[idx, 'Full_code_interval'] = 20  # 如果Full code從未發生過，可以用-1標記

In [20]:
df['Strength Arm'] = df[['Strength L Arm', 'Strength R Arm']].max(axis=1)
df['Strength Leg'] = df[['Strength L Leg', 'Strength R Leg']].max(axis=1)
df = df.drop(columns=['Strength L Arm', 'Strength R Arm', 'Strength L Leg', 'Strength R Leg'])

In [21]:
# 将处理后的 DataFrame 保存到 CSV 文件，并用 "NULL" 表示空值
df.to_csv(output_csv, index=False, na_rep="NULL")  # 使用 na_rep 指定 "NULL" 替代 NaN

In [22]:
# 读取 CSV 文件
df_original = pd.read_csv(f'{route}/number_2.5%.csv')
df_original = df_original.rename(columns={'FiO2': 'avg FiO2'})

# 选择必要的列
required_columns = ['stay_id', 'date', 'Vasopressor', 'avg FiO2', 'use_vent']

# 只保留必要的列
df = df_original[required_columns].copy()

# 转换日期列
df['date'] = pd.to_datetime(df['date'], errors='coerce')  # 确保日期列一致
df_original['date'] = pd.to_datetime(df_original['date'], errors='coerce')

# 确保数据按 stay_id 和 date 排序
df = df.sort_values(by=['stay_id', 'date'])

# 初始化标记
df['Consecutive_Vasopressor_Over3'] = 0
df['Consecutive_Vasopressor_Over7'] = 0
df['Consecutive_avg_FiO2_Over50'] = 0
df['Consecutive_avg_FiO2_Over60'] = 0


# 创建标记列
df['Is_Vasopressor_Active'] = df['Vasopressor'] == 1
df['Is_avg_FiO2_Over50'] = df['avg FiO2'] >= 50
df['Is_avg_FiO2_Over60'] = df['avg FiO2'] >= 60

# 使用矢量化计算分组
df['group_vasopressor'] = (df['Is_Vasopressor_Active'] != df['Is_Vasopressor_Active'].shift()).cumsum()
df['group_FiO2_50'] = (df['Is_avg_FiO2_Over50'] != df['Is_avg_FiO2_Over50'].shift()).cumsum()
df['group_FiO2_60'] = (df['Is_avg_FiO2_Over60'] != df['Is_avg_FiO2_Over60'].shift()).cumsum()

# 创建进度条，按 stay_id 分组
grouped = df.groupby('stay_id')
with tqdm(total=len(grouped), desc='Processing Groups') as pbar:
    for name, group in grouped:
        # 计算连续的天数
        group['Consecutive_Vasopressor_Days'] = group['Is_Vasopressor_Active'].astype(int).groupby(group['group_vasopressor']).cumsum()
        group['Consecutive_avg_FiO2_Over50_Days'] = group['Is_avg_FiO2_Over50'].astype(int).groupby(group['group_FiO2_50']).cumsum()
        group['Consecutive_avg_FiO2_Over60_Days'] = group['Is_avg_FiO2_Over60'].astype(int).groupby(group['group_FiO2_60']).cumsum()

        # 设置标记
        group.loc[group['Consecutive_Vasopressor_Days'] >= 3, 'Consecutive_Vasopressor_Over3'] = 1
        group.loc[group['Consecutive_Vasopressor_Days'] >= 7, 'Consecutive_Vasopressor_Over7'] = 1
        group.loc[group['Consecutive_avg_FiO2_Over50_Days'] >= 3, 'Consecutive_avg_FiO2_Over50'] = 1
        group.loc[group['Consecutive_avg_FiO2_Over60_Days'] >= 3, 'Consecutive_avg_FiO2_Over60'] = 1
        # 更新原 DataFrame
        df.update(group)
        # 更新进度条
        pbar.update(1)

# 清理临时列
columns_to_drop = ['Is_Vasopressor_Active', 'group_vasopressor', 'Consecutive_Vasopressor_Days', 
                   'Is_avg_FiO2_Over50', 'group_FiO2_50', 'Consecutive_avg_FiO2_Over50_Days', 
                   'Is_avg_FiO2_Over60', 'group_FiO2_60', 'Consecutive_avg_FiO2_Over60_Days']

# 删除存在的列
df.drop(columns_to_drop, axis=1, errors='ignore', inplace=True)

# 将处理后的数据与原始数据合并
result_df = df_original.merge(df, on=['stay_id', 'date'], how='left', suffixes=('', '_remove'))  # 确保列后缀

# 删除冗余列
columns_to_drop_after_merge = ['Vasopressor_remove', 'avg FiO2_remove','use_vent_remove']  # 删除重复列
result_df.drop(columns_to_drop_after_merge, axis=1, errors='ignore', inplace=True)



Processing Groups: 100%|██████████| 5870/5870 [00:59<00:00, 98.01it/s] 


In [23]:
result_df = result_df.rename(columns={'avg FiO2': 'FiO2'})
if 'icu_intime' in result_df.columns:
    result_df.drop('icu_intime', axis=1, inplace=True)
if 'icu_outtime' in result_df.columns:
    result_df.drop('icu_outtime', axis=1, inplace=True)

# 保存结果到 CSV
#result_df.to_csv(f'C:/Users/M1107171/MIMIC/清出來的資料/DNR/20240616/full_step1_5.csv',index = False)
result_df.to_csv(f'C:/Users/USER/M1326168/MIMIC/DNR/20250312/2/full_step1_5.csv',index = False)

# End

In [ ]:
"""
Vasopressor 測試
"""
#df = pd.read_csv(f'C:/Users/M1107171/MIMIC/清出來的資料/DNR/20240616/full_step1_5.csv')
df = pd.read_csv(f'C:/Users/USER/M1326168/MIMIC/DNR/20241002/full_step1_5.csv')    
df = df[['stay_id','Vasopressor','Consecutive_Vasopressor_Over3']]
distinct_stay_id = df['stay_id'].unique()

for stay_ids in tqdm(distinct_stay_id): 
    df_P = df[df['stay_id'] == stay_ids]
    if df_P['Vasopressor'].max() == 1:
        print(df_P)
        input("***********")

  0%|          | 0/9513 [00:00<?, ?it/s]

     stay_id  Vasopressor  Consecutive_Vasopressor_Over3
16  30007565            1                            0.0
17  30007565            1                            0.0
18  30007565            1                            1.0
19  30007565            1                            1.0
20  30007565            1                            1.0
21  30007565            0                            0.0
22  30007565            1                            0.0
23  30007565            1                            0.0
24  30007565            0                            0.0
25  30007565            0                            0.0
26  30007565            0                            0.0
27  30007565            0                            0.0
28  30007565            0                            0.0
29  30007565            0                            0.0
30  30007565            0                            0.0
31  30007565            0                            0.0
32  30007565            0      

  0%|          | 4/9513 [00:04<2:43:51,  1.03s/it]


KeyboardInterrupt: Interrupted by user

In [ ]:
"""
FiO2 測試
"""
#df = pd.read_csv(f'C:/Users/M1107171/MIMIC/清出來的資料/DNR/20240424/full_step1_5.csv')
df = pd.read_csv(f'C:/Users/USER/M1326168/MIMIC/DNR/20241002/full_step1_5.csv')
df = df[['stay_id','FiO2','Consecutive_avg_FiO2_Over50','Consecutive_avg_FiO2_Over60']]
distinct_stay_id = df['stay_id'].unique()

for stay_ids in tqdm(distinct_stay_id): 
    df_P = df[df['stay_id'] == stay_ids]
    if df_P['Consecutive_avg_FiO2_Over50'].max() == 1 or df_P['Consecutive_avg_FiO2_Over60'].max() == 1:
        print(df_P)
        input("***********")

In [ ]:
"""
Full_code_interval 測試
"""
#df = pd.read_csv(f'C:/Users/M1107171/MIMIC/清出來的資料/DNR/20240616/full_step1_5.csv')
df = pd.read_csv(f'C:/Users/USER/M1326168/MIMIC/DNR/20241002/full_step1_5.csv')
df = df[['stay_id','Full code','Full_code_interval']]
distinct_stay_id = df['stay_id'].unique()

for stay_ids in tqdm(distinct_stay_id): 
    df_P = df[df['stay_id'] == stay_ids]
    if df_P['Full code'].max() == 1:
        print(df_P)
        input("***********")
    